In [3]:
# Import necessary libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [4]:
# Set random seed for reproducibility
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
# Load dataset
data_path = r"C:\Users\HOME\OneDrive\Documents\Downloads\NonLinear_data.npy"
data = np.load(data_path, allow_pickle=True).item()
X, y = data['X'], data['labels']

print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")


Shape of X: (300, 2), Shape of y: (300,)


In [6]:
# Split data into train/validation/test sets
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=val_size, random_state=random_state, shuffle=is_shuffle
)
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle
)

print(f"Training size: {X_train.shape[0]}, Validation size: {X_val.shape[0]}, Test size: {X_test.shape[0]}")


Training size: 210, Validation size: 60, Test size: 30


In [7]:
# Normalize data
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


In [8]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create DataLoaders
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [9]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_dims, output_dims)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.output(x)
        return x

input_dims = X_train.shape[1]
hidden_dims = 128
output_dims = len(torch.unique(y_train))
model = MLP(input_dims, hidden_dims, output_dims).to(device)


In [10]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)


In [11]:
# Define accuracy function
def compute_accuracy(y_hat, y_true):
    _, y_hat = torch.max(y_hat, dim=1)
    correct = (y_hat == y_true).sum().item()
    accuracy = correct / len(y_true)
    return accuracy


In [12]:
# Train the model
epochs = 100
train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(epochs):
    model.train()
    train_loss, train_target, train_predict = 0.0, [], []
    for X_samples, y_samples in train_loader:
        X_samples, y_samples = X_samples.to(device), y_samples.to(device)
        optimizer.zero_grad()
        outputs = model(X_samples)
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_target.append(y_samples.cpu())
        train_predict.append(outputs.cpu())
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_target = torch.cat(train_target)
    train_predict = torch.cat(train_predict)
    train_accs.append(compute_accuracy(train_predict, train_target))
    
    model.eval()
    val_loss, val_target, val_predict = 0.0, [], []
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples, y_samples = X_samples.to(device), y_samples.to(device)
            outputs = model(X_samples)
            val_loss += criterion(outputs, y_samples).item()
            val_target.append(y_samples.cpu())
            val_predict.append(outputs.cpu())
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_target = torch.cat(val_target)
    val_predict = torch.cat(val_predict)
    val_accs.append(compute_accuracy(val_predict, val_target))

    print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_accs[-1]:.4f}')


Epoch 1/100 | Train Loss: 0.9386 | Val Loss: 0.8641 | Val Accuracy: 0.4833
Epoch 2/100 | Train Loss: 0.7497 | Val Loss: 0.8150 | Val Accuracy: 0.3833
Epoch 3/100 | Train Loss: 0.7150 | Val Loss: 0.7986 | Val Accuracy: 0.4833
Epoch 4/100 | Train Loss: 0.6890 | Val Loss: 0.7907 | Val Accuracy: 0.4500
Epoch 5/100 | Train Loss: 0.6696 | Val Loss: 0.7799 | Val Accuracy: 0.5167
Epoch 6/100 | Train Loss: 0.6541 | Val Loss: 0.7465 | Val Accuracy: 0.5000
Epoch 7/100 | Train Loss: 0.6455 | Val Loss: 0.7235 | Val Accuracy: 0.5333
Epoch 8/100 | Train Loss: 0.6158 | Val Loss: 0.7038 | Val Accuracy: 0.5167
Epoch 9/100 | Train Loss: 0.6165 | Val Loss: 0.6935 | Val Accuracy: 0.4833
Epoch 10/100 | Train Loss: 0.6096 | Val Loss: 0.6663 | Val Accuracy: 0.5667
Epoch 11/100 | Train Loss: 0.5769 | Val Loss: 0.6660 | Val Accuracy: 0.6000
Epoch 12/100 | Train Loss: 0.5962 | Val Loss: 0.6418 | Val Accuracy: 0.6167
Epoch 13/100 | Train Loss: 0.5557 | Val Loss: 0.6249 | Val Accuracy: 0.6333
Epoch 14/100 | Train 

In [13]:
# Evaluate the model on test data
model.eval()
with torch.no_grad():
    test_target, test_predict = [], []
    for X_samples, y_samples in test_loader:
        X_samples, y_samples = X_samples.to(device), y_samples.to(device)
        outputs = model(X_samples)
        test_target.append(y_samples.cpu())
        test_predict.append(outputs.cpu())
    test_target = torch.cat(test_target)
    test_predict = torch.cat(test_predict)
    test_accuracy = compute_accuracy(test_predict, test_target)
    print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9333
